# String token substitution

When a host application `resolve`s a string from the manager, it might be that the string contains placeholder tokens, which must be substituted with values held by the host application before the string can be used. Similarly, the host may publish an entity with a string property that contains placeholder tokens, which the manager must substitute before the string can be used.

This substitution mechanism is commonly used to represent dynamic content in an otherwise repeating pattern. For example, the path of images that make up an image sequence may be communicated as `path/to/image.{frame}`, expecting the host to substitute the frame number they desire, derived from their applications timeline or other selector.

Given that, typically, the host application and manager plugin are independently developed, this presents a few problems

* Hosts/managers need a way to find out if a string needs substitution at all.
* Hosts/managers need a shared understanding of how placeholder tokens are encoded within a string.
* Hosts/managers need to know which tokens are possible/allowed within a given string.
* Hosts/managers need the ability to substitute values into the placeholder tokens in a string.

OpenAssetIO solves these problems by a combination of documentation and a handy `substitute` utility function. The documentation of individual traits and their properties will tell hosts/managers whether they can place substitution tokens in a string, what tokens are allowed, and what values to map to each token. A common OpenAssetIO-dictated format for substitution tokens ensures that hosts/managers know how placeholders should be presented within strings. Once the host/manager has gathered the necessary values, they can use the `substitute` utility function to perform the substitution.


## Example - LocatableContent

Before anything else, a host needs to know if they need to account for substitutions. 

The LocatableContent trait has an `"isTemplated"` property, which, if true, indicates that the `"location"` property contains substitution tokens.

The `"isTemplated"` documentation says

> The URL in the location property contains variables (post decoding) that must be expanded before any loading is attempted.
>
> Variables use the OpenAssetIO syntax (eg: "image.{frame:04}.{view}.exr") see the OpenAssetIO documentation for more details.
>
> The following well-known variables are defined within the MediaCreation ecosystem, and must be used where applicable to any specific workflow:
> - frame: An integer frame number for the current time.
> - view: A string representing the current view (eg. "left").

This implies that a file name of `"image.{frame}.{view}.exr"` should be resolved to `"image.0001.left.exr"` if the frame number is 1 and the view is "left".

## The OpenAssetIO substitution syntax

In general, the substitution tokens are of the form `{variable:format}` where `variable` is the name of the variable and `format` is a format string. The `format` part is optional. 

For maximum cross language compatibility, only basic substitutions are officially supported. Specifically

* Substitution placeholders with no format string, e.g. `{frame}`.
* Placeholders for integers with zero padding, e.g. `{frame:04}`.

This may be revised in the future.

The syntax is compatible with the C++ [libfmt](https://fmt.dev/latest/syntax.html) library, which in turn is broadly compatible with C++20's [`std::format`](https://en.cppreference.com/w/cpp/utility/format/format) and Python [format strings](https://docs.python.org/3.9/library/string.html#formatspec). 

## The `substitute` utility function

The `substitute` utility function is a simple function that takes a string and a dictionary of substitutions, and returns a new string with the substitutions made. The dictionary should map variable names to their values.  Extra values will be ignored, and missing values is an error.

When constructing a string containing placeholder substitution tokens and sending that string over the OpenAssetIO API, then - unless you know otherwise - you should assume that the receiving end will use the OpenAssetIO `substitute` function. As such, you should ensure that the string is formatted according to the OpenAssetIO syntax, described above.



In [1]:
from resources import helpers  # helpers just for this notebook

from openassetio import utils

# A string with substitution tokens
template_path = "/mnt/show/sequences/image.{frame:04}.{view}.exr"

image_path = utils.substitute(template_path, {"frame": 12, "view": "right"})

helpers.display_result(image_path)

> **Result:**
> `/mnt/show/sequences/image.0012.right.exr`

If a token is missing, an exception will be thrown (this behaviour may be revised in future):

In [2]:
from openassetio import errors

try:
    utils.substitute(template_path, {"frame": 12})
except errors.InputValidationException as exc:
    helpers.display_result(str(exc))

> **Result:**
> `substitute(): failed to process the input string '/mnt/show/sequences/image.{frame:04}.{view}.exr': argument not found`

If an extra token mapping is provided, it will be ignored:

In [3]:
image_path = utils.substitute(template_path, {"frame": 12, "view": "right", "extra": "value"})

helpers.display_result(image_path)

> **Result:**
> `/mnt/show/sequences/image.0012.right.exr`

The `substitute` function is also available in C++, where the input dictionary is of type `openassetio::InfoDictionary`. In fact, the Python implementation is just the Python bindings to the C++ implementation, meaning that the Python dictionary must be coerced to an `InfoDictionary` internally.

If a value type is not supported by `InfoDictionary`, an exception will be thrown. For example, if we try to pass a nested dictionary as a value, an exception will be thrown:

In [4]:
try:
    utils.substitute(template_path, {"frame": 12, "view": "right", "nested": {"value": 1}})
except TypeError as exc:
    helpers.display_result(str(exc))

> **Result:**
> `substitute(): incompatible function arguments. The following argument types are supported:
    1. (input: str, substitutions: Dict[str, Union[bool, int, float, str]]) -> str

Invoked with: '/mnt/show/sequences/image.{frame:04}.{view}.exr', {'frame': 12, 'view': 'right', 'nested': {'value': 1}}`

Note that the error message shows what types are available for use as substitution variables - `bool`, `int`, `float` and `str`.